---

### *Title: Estimating the Parameters of the Bass Model*
Subtitle: Using Solver to Find the Best Fit for P, Q, and N
Content:
• How can we determine the optimal values for the Bass model parameters?
   - Use Solver to find P, Q, and N that minimize the Sum of Squared Errors (SSE)
      1. Input trial values for P, Q, and N
      2. Calculate predicted sales using the Bass equation
      3. Compute SSE between actual and predicted sales
   - Solver adjusts P, Q, and N to minimize SSE
      - Constraints: 0 ≤ P ≤ 1, 0 ≤ Q ≤ 1
   - Relative magnitudes of P and Q indicate importance of innovation vs. imitation


### *Title: Estimating Bass Model Parameters in Excel*
Content:
1. In cells E2:G2, input initial guesses for P, Q, and N. Name these cells p, q, and Nbar.

2. In cell D6, enter the Bass model formula: 
   =p*(Nbar-C5)+(q/Nbar)*C5*(Nbar-C5)
   Copy this formula down to cells D7:D21.

3. In cell E6, calculate the squared error between predicted and actual sales:
   =(B6-D6)^2 
   Copy this formula down to cells E7:E21.

4. In cell E3, calculate the Sum of Squared Errors (SSE) using:
   =SUM(E6:E21)

5. Open Solver and set it up as follows:
   - Set Objective: E3
   - To: Min
   - By Changing Variable Cells: $E$2:$G$2
   - Subject to the Constraints:
      - $E$2:$F$2 >= 0
      - $E$2:$F$2 <= 1
   - Select GRG Nonlinear as the Solving Method
   - Click Solve to find the optimal values of P, Q, and N that minimize SSE.



### *This Python code does the following:*

1. Imports necessary libraries (pandas, sqlite3, numpy, scipy.optimize)
2. Loads sales data from the Excel file "ColorTV.xlsx" into a pandas DataFrame
3. Creates an SQLite database "ColorTV.db" and a table "sales_data" from the DataFrame
4. Retrieves data from the SQLite table into a new DataFrame for further analysis
5. Defines the Bass model equation and the cost function (Sum of Squared Errors) to minimize
6. Sets initial parameter guesses and bounds for P, Q, and M
7. Uses scipy.optimize.minimize to find the optimal parameter values that minimize SSE
8. Prints out the optimal values of P, Q, and M
9. Closes the SQLite database connection

---

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
from scipy.optimize import minimize

# Load data from Excel into a pandas DataFrame
data = pd.read_excel('data/ColorTV.xlsx', sheet_name='Sheet1', usecols='A:C', skiprows=4, nrows=16)

# Connect to SQLite database and create a table from the DataFrame
conn = sqlite3.connect('data/ColorTV.db')
data.to_sql('sales_data', conn, if_exists='replace', index=False)

# Retrieve data from SQLite into a new DataFrame
query = 'SELECT * FROM sales_data'
df = pd.read_sql(query, conn)

# Define the Bass model equation
def bass_model(t, p, q, m):
    a = p + q
    b = p / m
    c = np.exp(-(a * t))
    return m * (((a**2)/p) * c) / (1 + ((q/p) * c))**2

# Define the cost function to minimize (Sum of Squared Errors)
def cost_function(params):
    p, q, m = params
    y_pred = bass_model(df['t'], p, q, m)
    y_true = df['n(t)']
    sse = np.sum((y_true - y_pred)**2)
    return sse

# Set initial parameter guesses and bounds
initial_params = [0.03, 0.38, 100]
bounds = [(0, 1), (0, 1), (0, None)]

# Use scipy.optimize.minimize to find optimal parameters
result = minimize(cost_function, initial_params, bounds=bounds, method='L-BFGS-B')
p, q, m = result.x

# Print optimal parameter values
print(f'Optimal P: {p:.3f}')  
print(f'Optimal Q: {q:.3f}')
print(f'Optimal M: {m:.3f}')

# Close the SQLite connection
conn.close()

---


### *Title: Forecasting New Product Sales with the Bass Model* ###
Subtitle: Leveraging Adjacent Categories to Predict Adoption
Content: 
- **Analogous products** from adjacent sectors can guide new product forecasts
  - Find a similar, mature product (e.g., color TV for DIRECTV)
  - Use the analog's P and Q values with estimated N for the new product
- **Intentions data** helps estimate initial adoption but often overstates demand
  - Adjust using factors like affordability and availability 
  - *Apply a research-based fractional correction (k) to the stated purchase intent percentage*
- **Goal Seek** in Excel reverse engineers key Bass model parameters
  - Set N_bar so the model matches your adjusted 1-year adoption estimate
 

### *Title: The Power of Empirical Generalization* ###
Subtitle: Discovering Patterns that Transcend Individual Circumstances
Content:
- **Empirical generalization** identifies consistent patterns across contexts
  - Described by mathematical, graphical, or symbolic methods
  - Enables insights and predictions based on a small set of core drivers
- **The Bass model** is a prime example in new product forecasting
  - P (coefficient of innovation) and Q (coefficient of imitation) often hold steady
  - *Patterns of early adoption and word-of-mouth spread repeat across products*
- **Managers** can harness this power for data-driven decisions
  - Recognize the prevalence and potential of empirical regularities
  - Build robust models on a backbone of shared dynamics



### *Title: Using Excel to Forecast DIRECTV Adoption with the Bass Model* ###
Content:
1. Enter 0 in cells F7 and G7 for the initial number of DIRECTV subscribers.
2. In cell F8, enter the formula =(1/12)*(p*(Nbar-G7)+q*G7*(Nbar-G7)/Nbar) and copy down to F55. This estimates new subscribers each month using Equation 1, adjusting P and Q to monthly values.  
3. In cell G8, enter the formula =G7+F8 and copy down to G55 to calculate cumulative subscribers by adding the previous total to the current month's new subscribers.
4. Use Excel's Goal Seek (Data > What-If Analysis > Goal Seek) to find the Nbar value that makes the 1-year subscriber estimate (G19) match an exogenous projection (e.g., 1.32 million). 
  - Set cell: $G$19 
  - To value: 1.32
  - By changing cell: $F$4
5. The value of Nbar that matches the 1-year estimate (e.g., 21.55 million for 1.32 million subscribers) can now be used in the Bass model to forecast subscribers at later timepoints (e.g., G55 for a 4-year estimate).


---


This Python script:
1. Loads analogous product parameters (p, q) and an exogenous 1-year subscriber estimate 
2. Reads the Excel data into a pandas DataFrame
3. Creates an SQLite3 database and populates a 'subscribers' table with the Excel data
4. Defines functions to:
   - Calculate Bass model estimates for given parameters
   - Calculate the sum of squared errors (SSE) between estimates and actual data
5. Uses iterative optimization to find the Nbar that minimizes SSE, matching the 1-year estimate
6. Queries the database for the 4-year subscriber forecast based on the optimized Nbar
7. Prints the optimal Nbar and 4-year forecast  


---

In [ ]:
import pandas as pd
import sqlite3

# Load analogous product parameters 
p = 0.059 
q = 0.1463

# Load exogenous 1-year estimate of new product subscribers
subs_1yr = 1.32 

# Load Excel data into a DataFrame 
df = pd.read_excel('data/DIRECTV.xlsx', sheet_name='Sheet1', usecols='A:G', nrows=56)

# Initialize a SQLite3 database and connection
conn = sqlite3.connect('data/DIRECTV.db')
cur = conn.cursor()

# Create a subscribers table and populate it with Excel data
cur.execute('''CREATE TABLE IF NOT EXISTS subscribers
               (t INTEGER, actual REAL, cumulative REAL)''')
          
for index, row in df.iterrows():
    t = index - 6
    actual = row['F'] 
    cumulative = row['G']
    cur.execute("INSERT INTO subscribers VALUES (?,?,?)", (t, actual, cumulative))

conn.commit()

# Function to calculate Bass model estimate for a given t and Nbar
def bass_model(t, Nbar, actual, cumulative):
    return (1/12) * (p * (Nbar - cumulative) + q * actual * (Nbar - cumulative) / Nbar)

# Function to calculate SSE between Bass model estimates and DIRECTV data
def calc_sse(Nbar):
    cur.execute("SELECT * FROM subscribers")
    data = cur.fetchall()
    
    sse = 0
    for row in data:
        t = row[0]
        actual = row[1]
        cumulative = row[2]
        
        estimate = bass_model(t, Nbar, actual, cumulative)
        sse += (estimate - actual)**2
        
    return sse

# Optimize Nbar to match 1-year subscriber estimate    
Nbar = 1
step = 10000000

while True:
    sse_curr = calc_sse(Nbar)
    sse_next = calc_sse(Nbar + step)
    
    if sse_next > sse_curr:
        step /= 2
    else:
        Nbar += step
        
    if step < 0.001:
        break
        
print(f"Optimal Nbar: {Nbar:.2f}")

cur.execute("SELECT cumulative FROM subscribers WHERE t = 48")
subs_4yr = cur.fetchone()[0]
print(f"4-year subscriber forecast: {subs_4yr:.2f} million")

conn.close()

---

### *Title: How Uncertain are New Product Sales?* ###
Subtitle: Combining the Bass Model with Monte Carlo Simulation  
Content:
- **New product sales** are highly uncertain
  - Difficult to predict market size and adoption patterns
  - Traditional point forecasts may give false sense of precision
- **Monte Carlo simulation** models uncertain quantities  
  - Represents inputs as probability distributions rather than single values
  - *Generates range of possible outcomes to capture inherent uncertainty*
- **Bass model parameters** can be treated as random variables
  - Market potential (m): often modeled as a normal distribution
  - *Coefficients of innovation (p) and imitation (q): drawn from analogous products*
 

### *Title: Illuminating the Darkness of Uncertainty* ###
Subtitle: Why Simulation Beats Point Forecasts for New Products
Content:
- **Point forecasts** give illusion of certainty and precision
  - Single "best guess" values for market size, adoption rates, etc.
  - But reality is often far more uncertain, especially for new products
- **Simulation** embraces and quantifies inherent uncertainty
  - Presents a realistic range of possibilities 
  - *Helps identify key drivers of variation in outcomes*
  - Facilitates robust decision making under uncertainty
- **Combining simulation with the Bass model** is especially powerful
  - Acknowledges our imperfect knowledge of key parameters
  - *Leverages data from analogous products to constrain estimates*
  - Yields actionable insights for managing new product launches


### *Title: Simulating New Product Sales with the Bass Model in Excel* ###
Content:
1. In cell C4, simulate potential market size (simNbar) with the formula =ROUND(NORMINV(RAND(),C2,C3),0), assuming a normal distribution with mean in C2 and standard deviation in C3.
2. In cell H1, choose a random Bass parameter scenario (1-12) with =RANDBETWEEN(1,12).
3. In cell H2, look up the P value for the chosen scenario with =VLOOKUP(scenario, B6:D17,2).
4. In cell H3, look up the Q value with =VLOOKUP(scenario, B6:D17,3).
5. In cell I6, enter 0 for initial cumulative sales. 
6. In I7, calculate cumulative sales with =SUM($H$6:H7). Copy this formula down to I18.
7. In H7, calculate year 1 sales with =simp*(simNbar-I6)+simq*I6*(simNbar-I6)/simNbar. Copy this formula down to H18.
8. Set up a data table to simulate 1,000 iterations:
   - In cells F22, G22, and H22, reference 1-year (I7), 5-year (I11), and 10-year (I16) cumulative sales.
   - Select E22:H1022 and go to Data > What-If Analysis > Data Table.
   - Leave the Row Input cell blank and select any blank cell for the Column Input. 
9. Calculate average sales in F20 with =AVERAGE(F23:F1022). Copy this to G20:H20. 
10. Calculate standard deviations in F21 with =STDEV(F23:F1022). Copy this to G21:H21.

The data table simulates 1,000 possible sales trajectories, capturing uncertainty in market potential and Bass parameters. The averages and standard deviations of 1-, 5-, and 10-year sales provide a realistic range of outcomes to inform decision making.


---


### *This Python script:* ###
1. Reads Bass model parameters and scenarios from the Excel file into a pandas DataFrame.
2. Creates an SQLite database and populates a 'scenarios' table with the Bass parameters.
3. Defines functions to:
   - Simulate market potential from a normal distribution.
   - Retrieve Bass parameters for a randomly selected scenario from the database.
   - Simulate the Bass model for given parameters and market potential.
   - Run n iterations of the Bass model simulation.
4. Simulates 1000 iterations of the Bass model for 10 periods.
5. Prints the average and standard deviation of cumulative sales for periods 1, 5, and 10.


---

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# Read Bass parameters and scenarios from Excel
df = pd.read_excel('data/Basssim.xlsx', sheet_name='Sheet1', usecols='B:D', skiprows=5, nrows=12) 

# Initialize SQLite database and connection
conn = sqlite3.connect('data/Basssim.db')

# Create scenarios table and insert Bass parameters
df.to_sql('scenarios', conn, if_exists='replace', index=False)

# Simulate market potential from normal distribution
def simulate_market_potential(mean, sigma):
    return round(np.random.normal(mean, sigma))

# Retrieve Bass parameters for a random scenario
def get_bass_params(conn):
    query = '''
        SELECT p, q 
        FROM scenarios
        WHERE "Product" = (SELECT "Product" 
                           FROM scenarios
                           ORDER BY RANDOM()
                           LIMIT 1)
    '''
    params = pd.read_sql_query(query, conn).values[0]
    return params[0], params[1]

# Simulate Bass model for given parameters and market potential
def bass_model(p, q, market_potential, num_periods):
    sales = np.zeros(num_periods)
    cum_sales = np.zeros(num_periods)
    
    for i in range(num_periods):
        if i == 0:
            sales[i] = p * market_potential
        else:
            sales[i] = (p + q * cum_sales[i-1] / market_potential) * (market_potential - cum_sales[i-1])
        cum_sales[i] = cum_sales[i-1] + sales[i]
    
    return sales, cum_sales

# Simulate n iterations of the Bass model
def simulate_bass(n, mean, sigma, num_periods):
    results = pd.DataFrame(columns=[f'Period {i+1}' for i in range(num_periods)])
    
    for _ in range(n):
        market_potential = simulate_market_potential(mean, sigma)
        p, q = get_bass_params(conn)
        _, cum_sales = bass_model(p, q, market_potential, num_periods)
        results.loc[len(results)] = cum_sales
        
    return results

# Simulate 1000 iterations for 10 periods
market_potential_mean = 100000
market_potential_sigma = 20000
num_periods = 10
num_iterations = 1000

simulation_results = simulate_bass(num_iterations, market_potential_mean, market_potential_sigma, num_periods)

# Print average and standard deviation of cumulative sales for periods 1, 5, and 10
for period in [1, 5, 10]:
    print(f"Period {period}:")
    print(f"  Average Sales: {simulation_results[f'Period {period}'].mean():.0f}")
    print(f"  Standard Deviation: {simulation_results[f'Period {period}'].std():.0f}")

conn.close()

---

### *Title: Adstock Model: Accounting for Advertising Lag Effects* 
Subtitle: How do ads affect present and future sales?
Content:
- **Key Idea**: A fraction (λ) of previous ad stock is retained each period
  - λ = 0.8: Ad from 1 period ago has 80% effect of current ad
  - λ = 0.8: Ad from 2 periods ago has 0.8^2 = 51.2% effect of current ad
- **Adstock Level**: Computed using equations
  - *Quarter 1*: ADSTOCK = λ * INITIAL ADSTOCK + QUARTER 1 ADS 
  - *Quarter T*: ADSTOCK = λ * QUARTER (T-1) ADSTOCK + QUARTER T ADS
- **Sales Forecast**: Predicted Sales = (CONST * (TREND^Quarter# + ADEFFECT * ADSTOCK) * (PRICE)^(-elasticity) * (Seasonal Index)


### *Title: Interpreting Adstock Model Parameters*
Subtitle: What insights can we glean from optimized Adstock models? 
Content:  
- **Sales Trend**: % increase in sales per period
  - E.g. 9.7% quarterly growth rate
- **Ad Effectiveness Decay**: % of ad power lost each period   
  - E.g. 17% lost quarterly if λ = 0.83
- **Price Elasticity**: % demand change for 1% price change
  - E.g. -1.49 elasticity means 1% price hike → 1.49% demand drop
- **Seasonality Indices**: % deviation from average period sales  
  - E.g. Q4 sales 67% above avg, Q2 30% below (indices 1.67 vs 0.70)


### *Title: Implementing the Adstock Model in Excel*
Content:
1. In cell F6, use Equation 1 to compute quarter 1's Adstock: 
   - Formula: =E6+initialadstock*lambda
2. In cells F7:F29, use Equation 2 to compute remaining quarters' Adstock:
   - Formula: =E7+lambda*F6 (copy down)  
3. In cells H6:H29, use Equation 3 to compute sales forecasts:
   - Formula: =(const*(trend)^D6+adeffect*F6)*VLOOKUP(C6,season,2)*(G6)^(-elasticity)
4. In cells J6:J29, compute each quarter's absolute percentage error:  
   - Formula: =ABS(I6-H6)/I6
5. In cell I4, compute MAPE:
   - Formula: =AVERAGE(J6:J29)
6. Use Solver (per Figure 32-1) to minimize MAPE by changing model parameters:
   - Objective: Set MAPE (cell I54) to Min
   - Variable Cells: const, trend, lambda, adeffect, elasticity, initialadstock  
   - Constraints: parameter bounds, seasonality sum to 1
   - Select GRG Nonlinear solving method 
7. Interpret optimized parameters (per Figure 34-2)



### *The Python code follows these steps:*
1. Load the Excel data into a pandas DataFrame
2. Create a SQLite database and connection  
3. Write the DataFrame to a table in SQLite
4. Query the full data set from SQLite into a new DataFrame
5. Perform Adstock calculations using pandas shift() and fillna()
6. Create seasonal dummy variables via get_dummies()
7. Set up expanded Adstock sales forecast equation 
8. Define MAPE objective function 
9. Use Scipy's minimize() to find optimal parameters
   - Provide initial guesses, parameter bounds, seasonality constraint
   - Use SLSQP solver for constrained optimization
10. Print the minimized MAPE and optimal parameter values


---

In [ ]:
import pandas as pd
import sqlite3
import numpy as np

# Load data from Excel into pandas DataFrame  
data = pd.read_excel('Adstock.xlsx')

# Create SQLite database and connection
conn = sqlite3.connect('adstock.db') 
data.to_sql('adstock', conn, index=False)

# Query data from SQLite
sql = '''
  SELECT *
  FROM adstock
'''
df = pd.read_sql(sql, conn)

# Adstock calculations
df['adstock'] = (df['ads'].shift(1).fillna(0) + 
                initialadstock) * lambda
df_in = df[['qtr','ads','price','actual']]  

# Seasonal dummies
df_in = pd.concat([df_in, pd.get_dummies(df.qtr)], axis=1) 

# Expended sales forecast equation  
sales_forecast = const * (trend**np.arange(len(df))) + adeffect*df.adstock
sales_forecast *= (df.price ** -elasticity) * (df.loc[:,1:4] * season).sum(1) 
  
# MAPE objective function  
def adstock_mape(x):
    prediction = forecast_func(x, df)
    return np.mean(np.abs((df.actual - prediction) / df.actual)) * 100

# Use Scipy optimize to minimize MAPE
from scipy.optimize import minimize
x0 = [1,1.04,.7,1,500,35000,1.5,.7,.7,1.5]  # Initial parameter guesses
bnds = ((0, None),(1,1.5),(0,1),(0,None),(0,1e5),(0,1e6),(0,10),
        (0,1),(0,1),(1,None)) 
cons = [{'type':'eq', 'fun': lambda x: np.sum(x[6:10])-4}]  # Season sum to 1
res = minimize(adstock_mape, x0, method='SLSQP', bounds=bnds, constraints=cons)
print(f"Optimal Parameters: {res.x}")
print(f"Minimized MAPE: {res.fun:.2f}%")

---

### *Title: Lilien-Kotler-Moorthy Ad Effectiveness Model*
Subtitle: How do advertising levels, changes, and carryover impact sales?
Content:
- **Sales Response Function**: Qt = a + λQt-1 + bLN(At) + cmax(0,ΔAt)
  - Qt: Period t sales 
  - At: Period t advertising
  - ΔAt: % change in ads vs prior period
- **Key Model Features**:
  - *Diminishing returns* to advertising spend via LN(At)
  - *Advertising carryover* effect via lagged sales term λQt-1
  - *Pulse effects* from ad changes via cmax(0,ΔAt)


### *Title: Comparing Ad Response Models*
Subtitle: What are the key differences between Adstock and LKM? 
Content:
- **Adstock Model**: 
  - *Geometric decay* of ad effect over time
  - Ad stock *level* drives sales  
  - Excel-friendly *multiplicative* model form
- **LKM Model**:
  - *Lagged sales term* captures ad carryover
  - *Log* and *max* terms add flexibility
  - *Additive* (not multiplicative) model structure
- **Commonalities**: 
  - Quantify ad *carryover* and *diminishing returns*
  - Require *unconstrained optimization* to fit


### *Title: Fitting the LKM Model in Excel*
Content:
1. In cells F8:F42, compute ΔAt for each month: 
   - Formula: =(D8-D7)/D7
2. In cells G8:G42, compute sales forecasts using LKM equation:
   - Formula: =a+lambda*E8+b*LN(D8)+c_*MAX(0,F8) 
3. In cells H8:H42, compute squared errors:
   - Formula: =(G8-C8)^2
4. In cell H5, compute SSE:   
   - Formula: =SUM(H8:H42)
5. In cells I8:I42, compute forecast errors:
   - Formula: =C8-G8  
6. Use Solver (per Fig 34-4) to minimize SSE by changing a, lambda, b, c_:
   - Objective: Set cell H5 to Min  
   - Variable Cells: a, lambda, b, c_ 
   - Constraints: a>=0, 0<=lambda<=1, b>=0, c_>=-10 
   - Select GRG Nonlinear engine
7. Interpret fitted model (per steps in passage)


### *The Python code follows these steps:*
1. Load data from Excel, skipping headers and selecting relevant columns
2. Create SQLite database and write data to a table
3. Query full data set from SQLite into a DataFrame 
4. Create lagged variables sales (t-1) and ΔAt
5. Define RSS objective function based on LKM equation
6. Provide initial parameter guesses and bounds
7. Use Scipy's minimize() to find optimal parameters  
   - Nelder-Mead solver for unconstrained problem
   - Print optimal values of a, b, c, λ
8. Could add more code to analyze fit, plot actuals vs predicted, etc.


---

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
from scipy.optimize import minimize

# Load data from Excel into pandas DataFrame
data = pd.read_excel('addata.xls', skiprows=5, nrows=37, 
                     usecols='B:D', names=['month','sales','adv'])

# Create SQLite database and connection 
conn = sqlite3.connect('addata.db')
data.to_sql('addata', conn, index=False)

# Query data from SQLite  
query = '''
  SELECT * 
  FROM addata
'''
df = pd.read_sql(query, conn)

# Lag variables
df['lagsales'] = df.sales.shift(1)  
df['deltaa'] = df.adv.pct_change()

# Residual sum of squares (RSS) objective function
def rss_lkm(params):
    a, b, c, lam = params
    sales_hat = (a + lam*df.lagsales + b*np.log(df.adv) + 
                 c*np.maximum(0,df.deltaa)) 
    return np.sum((df.sales[1:] - sales_hat[1:])**2)

# Set initial param guesses and bounds
guesses = [1, 1, 0, 0.5] 
bounds = [(0,None), (0,None), (None,None), (0,1)]

# Use minimize() to estimate optimal params 
results = minimize(rss_lkm, guesses, method='L-BFGS-B', bounds=bounds)
print(f"Optimal a = {results.x[0]:.3f}")  
print(f"Optimal b = {results.x[1]:.3f}")
print(f"Optimal c = {results.x[2]:.3f}") 
print(f"Optimal λ = {results.x[3]:.3f}")

---

### *Title: Optimizing Advertising Spend*
Subtitle: Continuous vs. Pulsing Strategies
Content:  
- How can we maximize profit through advertising?
  - **Continuous spending:** constant monthly investment
  - **Pulsing:** alternating high/low ad levels
- Key factors:
  - *Planning horizon length*
  - *Problem parameters*
- In practice: 
  - Continuous spending often optimal


### *Title: Factors Influencing Optimal Ad Strategy*
Subtitle: Planning Horizon and Problem Parameters
Content:
- What determines the best approach?
  - **Planning horizon length**
    - *Longer → favors continuous*
    - *Shorter → may favor pulsing*  
  - **Problem parameters** 
    - *Market factors, objectives, constraints*
- Analyzing unique situational variables is key



### *Lab Slide 1 - Excel Steps:*
1. Input assumptions in cells G1:G5 
   - 30% profit margin 
   - $1,000 unit price
   - $100 per ad cost
   - Month 1 sales: 12 units
   - Month 1 ads: 9
2. In F9, calculate profit: 
   =price*C9*profit_margin-100*D9
   Copy to F10:F43 for months 2-36
3. In C9, forecast sales:
   =a+lambda*C8+b*LN(D9)+c_*MAX(0,E9)  
   Copy to C10:C43
4. Use Solver to maximize total profit (F44) 
   By changing ad levels in D10:D43
   Constrain each month's ads >= 0.10


---

In [ ]:
import pandas as pd
import sqlite3

# Load data from Excel 
data = pd.read_excel('addata.xlsx', sheet_name='optimize', usecols='G1:G5', nrows=5)

# Create database connection
conn = sqlite3.connect('addata.db')

# Write data to SQLite table
data.to_sql('assumptions', conn, index=False)

# Query data from SQLite
cur = conn.cursor()
cur.execute('''
  SELECT *
  FROM assumptions
''')

assumptions = cur.fetchall()
print(assumptions)

# Placeholder for optimization code
def optimize_ads(assumptions):
  # Implement optimization logic
  
  # Return optimized ad levels
  return optimal_ads

# Get assumptions from SQLite
profit_margin, price, ad_cost, month1_sales, month1_ads = assumptions[0]

# Run optimization
optimal_ads = optimize_ads(assumptions)

print(f"Optimal ad levels: {optimal_ads}")

# Close database connection
conn.close()